# Implémentation d'un circuit "bruyant" - Noisy circuit implementation

Ce tutoriel consiste à simuler des circuits bruyants à l'aide des fonctionnalités intégrées à PennyLane. 

La première partie consiste à étudier les canaux bruyants et les matrices de densité, puis la seconde partie consiste à simuler des circuits bruyants. 
La troisième partie consiste à utiliser les fonctionnalités de PennyLane pour calculer les gradients des canaux bruités afin d'optimiser les paramètres de bruit dans un circuit.

Le bruit est une transformation indésirable qui modifie la sortie prévue d'un calcul quantique. Il existe deux catégories de bruit :

- Le *bruit cohérent* : il provient de dispositifs imparfaitement calibré qui n'appliquent pas exactement les portes souhaitées, par exemple en appliquant une rotation d'angle $\phi + \epsilon$ ua lieu de l'angle $\phi$.

- Le *bruit incohérent* : il provient d'une interaction entre l'ordinateur quantique et l'environnement, qui induit des distributions de probabilités sur les états du qubits en sortie (états mixtes). La sortie est donc toujours aléatoire.

Les états mixtes sont décrits par des *matrices de densité*, qui permettent de décrire la distribution de probabilités pour un état quantique.

Commencons par afficher la matrice de densité de l'état de Bell $|\phi\rangle = \frac{1 }{\sqrt(2)}  * (|00\rangle + |11\rangle)$.

\- \- \-


This tutorial is about simulating noisy circuits using the features built into PennyLane.

The first part consists in studying the noisy channels and the density matrices, then the second part consists in simulating noisy circuits.
The third part consists in using the functionalities of PennyLane to calculate the gradients of the noisy channels in order to optimize 
the noise parameters in a circuit.


Noise is an unwanted transformation that alters the intended output of a quantum computation. There are two categories of noise:

- The *coherent noise*: it comes from imperfectly calibrated devices which do not apply exactly the desired gates, for example by applying a rotation of angle $\phi + \epsilon$ instead of angle $\phi$.

- The *incoherent noise*: it comes from an interaction between the quantum computer and the environment, which induces probability distributions on the states of the output qubits (mixed states). The output is therefore always random.

Mixed states are described by *density matrices*, which allow to describe the probability distribution for a quantum state.

Let's start by displaying the Bell state density matrix $|\phi\rangle = \frac{1 }{\sqrt(2)} * (|00\rangle + |11\rangle)$.



In [84]:
import pennylane as qml
from pennylane import numpy as np

In [85]:
qml.about()

Name: PennyLane
Version: 0.25.1
Summary: PennyLane is a Python quantum machine learning library by Xanadu Inc.
Home-page: https://github.com/XanaduAI/pennylane
Author: 
Author-email: 
License: Apache License 2.0
Location: c:\users\maell\appdata\local\programs\python\python310\lib\site-packages
Requires: appdirs, autograd, autoray, cachetools, networkx, numpy, pennylane-lightning, retworkx, scipy, semantic-version, toml
Required-by: PennyLane-Lightning, PennyLane-SF

Platform info:           Windows-10-10.0.22000-SP0
Python version:          3.10.5
Numpy version:           1.23.1
Scipy version:           1.8.1
Installed devices:
- default.gaussian (PennyLane-0.24.0)
- default.mixed (PennyLane-0.24.0)
- default.qubit (PennyLane-0.24.0)
- default.qubit.autograd (PennyLane-0.24.0)
- default.qubit.jax (PennyLane-0.24.0)
- default.qubit.tf (PennyLane-0.24.0)
- default.qubit.torch (PennyLane-0.24.0)
- lightning.qubit (PennyLane-Lightning-0.24.0)


(PennyLane _default.mixed_ permet de fournir une prise en charge native des états mixtes et de simuler des calculs bruyants)

\- \- \-

(PennyLane _default.mixed_ helps provide native support for mixed states and simulate noisy computations)

In [86]:
dev = qml.device('default.mixed', wires=2)

@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))




L'appareil stocke l'état de sortie sous forme de matrice de densité. 
Dans ce cas, la matrice de densité est égale à $|\phi\rangle\langle\phi|$ où $|\phi\rangle = \frac{1 }{\sqrt(2)}  * (|00\rangle + |11\rangle)$.

\- \- \-

The device stores the output state as a density matrix.
In this case, the density matrix is ​​equal to $|\phi\rangle\langle\phi|$ where $|\phi\rangle = \frac{1 }{\sqrt(2)} * (|00\rangle + |11\rangle)$.

In [87]:
print(f"Output state is = \n{np.real(dev.state)}")

Output state is = 
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Le bruit incohérent est modélisé par des *canaux quantiques*, représenté mathématiquement par les opérateurs de Kraus ($K_i$) tels que $\sum _i K_i ^\dagger K_i = I$. Ainsi, pour un état initial $\rho$, l'état de la sortie après l'action d'un canal $\phi$ est égale à :

$\phi(\rho) = \sum _i K_i \rho K_i^\dagger$


Plus généralement, l'action d'un canal quantique peut être interpretée comme appliquant une transformation correspondant à l'opérateur de Kraus $K_i = \frac{1}{p_i}K_i \rho K_i^\dagger$ avec une probabilité $p_i = Tr[K_i \rho K_i^\dagger]$.

Par exemple, considérons le canal qui décrit l'inversion de l'état d'un qubit (porte X) avec une probabilité $p$, et qui laisse l'état du qubit inchangé avec une probabilité $1-p$. Ses opérateurs de Kraus sont :

$K_0 = \sqrt{1 - p}  \left( {\begin{array}{cc}
    1 & 0 \\
    0 & 1 \\
  \end{array} } \right)$

$K_1 = \sqrt{p}  \left( {\begin{array}{cc}
    0 & 1 \\
    1 & 0 \\
  \end{array} } \right)$


  (implémenté à l'aide de _qml.BitFlip_)


  \- \- \-

  Incoherent noise is modeled by *quantum channels*, mathematically represented by Kraus operators ($K_i$) such as $\sum _i K_i ^\dagger K_i = I$. Thus, for an initial state $\rho$, the state of the output after the action of a channel $\phi$ is equal to:

$\phi(\rho) = \sum _i K_i \rho K_i^\dagger$


More generally, the action of a quantum channel can be interpreted as applying a transformation corresponding to the Kraus operator $K_i = \frac{1}{p_i}K_i \rho K_i^\dagger$ with probability $p_i = Tr[K_i \rho K_i^\dagger]$.

For example, consider the channel that describes the reversal of the state of a qubit (gate X) with probability $p$, and leaves the qubit state unchanged with probability $1-p$. Its Kraus Operators are:

$K_0 = \sqrt{1 - p} \left( {\begin{array}{cc}
    1 & 0 \\
    0 & 1 \\
  \end{array} } \right)$

$K_1 = \sqrt{p} \left( {\begin{array}{cc}
    0 & 1 \\
    1 & 0 \\
  \end{array} } \right)$


  (implemented using _qml.BitFlip_)


In [88]:
@qml.qnode(dev)
def bitflip_circuit(p):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.BitFlip(p, wires=0)
    qml.BitFlip(p, wires=1)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))


ps = [0.001, 0.01, 0.1, 0.2]
for p in ps :
    print(f"La sortie du QNode pour une probabilité de bitflip de {p} est {bitflip_circuit(p):.4f}")
    print(f"L'état mixte du QNode pour une probabilité de bitflip de {p} is \n{np.real(dev.state)}")


La sortie du QNode pour une probabilité de bitflip de 0.001 est 0.9960
L'état mixte du QNode pour une probabilité de bitflip de 0.001 is 
[[0.499001 0.       0.       0.499001]
 [0.       0.000999 0.000999 0.      ]
 [0.       0.000999 0.000999 0.      ]
 [0.499001 0.       0.       0.499001]]
La sortie du QNode pour une probabilité de bitflip de 0.01 est 0.9604
L'état mixte du QNode pour une probabilité de bitflip de 0.01 is 
[[0.4901 0.     0.     0.4901]
 [0.     0.0099 0.0099 0.    ]
 [0.     0.0099 0.0099 0.    ]
 [0.4901 0.     0.     0.4901]]
La sortie du QNode pour une probabilité de bitflip de 0.1 est 0.6400
L'état mixte du QNode pour une probabilité de bitflip de 0.1 is 
[[0.41 0.   0.   0.41]
 [0.   0.09 0.09 0.  ]
 [0.   0.09 0.09 0.  ]
 [0.41 0.   0.   0.41]]
La sortie du QNode pour une probabilité de bitflip de 0.2 est 0.3600
L'état mixte du QNode pour une probabilité de bitflip de 0.2 is 
[[0.34 0.   0.   0.34]
 [0.   0.16 0.16 0.  ]
 [0.   0.16 0.16 0.  ]
 [0.34 0.   0.

### Canal d'erreur dépolarisant - Depolarizing error channel

Implémentons un circuit soumis à un canal d'erreur dépolarisant :
- Pour $p = 0$, le canal sera un canal d'identité, c'est-à-dire un canal sans bruit.
- Pour $p = \frac{3}{4}$, le canal sera un canal entièrement dépolarisant.
- Pour $p = 1$, le canal sera un canal d'erreur de Pauli uniforme.

Le canal dépolarisant est décrit par le système de Kraus suivant :


$K_0 =  \sqrt{1 - p}  \left( {\begin{array}{cc}
    1 & 0 \\
    0 & 1\\
  \end{array} } \right)$
$K_1 =  \sqrt{\frac{p}{3}}  \left( {\begin{array}{cc}
    0 & 1\\
    1 & 0 \\
  \end{array} } \right)$
  $K_2 =  \sqrt{\frac{p}{3}}  \left( {\begin{array}{cc}
    0 & -i \\
    i & 0\\
  \end{array} } \right)$
  $K_3 =  \sqrt{\frac{p}{3}}  \left( {\begin{array}{cc}
    1 & 0 \\
    0 & -1\\
  \end{array} } \right)$

  \- \- \-



  Let's implement a circuit subject to a depolarizing error channel:
- For $p = 0$, the channel will be an identity channel, ie a noiseless channel.
- For $p = \frac{3}{4}$, the channel will be a fully depolarizing channel.
- For $p = 1$, the channel will be a uniform Pauli error channel.

The depolarizing channel is described by the following Kraus system:


$K_0 = \sqrt{1 - p} \left( {\begin{array}{cc}
    1 & 0 \\
    0 & 1\\
  \end{array} } \right)$
$K_1 = \sqrt{\frac{p}{3}} \left( {\begin{array}{cc}
    0 & 1\\
    1 & 0 \\
  \end{array} } \right)$
  $K_2 = \sqrt{\frac{p}{3}} \left( {\begin{array}{cc}
    0 & -i \\
    i & 0\\
  \end{array} } \right)$
  $K_3 = \sqrt{\frac{p}{3}} \left( {\begin{array}{cc}
    1 & 0 \\
    0 & -1\\
  \end{array} } \right)$

In [91]:

dev_depolarizing = qml.device('cirq.simulator', wires=4)




DeviceError: Device does not exist. Make sure the required plugin is installed.

In [53]:
@qml.qnode(dev_depolarizing)
def Depolarizing_circuit(p):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    qml.Hadamard(wires=3)
    qml.DepolarizingChannel(p, 0, False, None)
    qml.DepolarizingChannel(p, 2, False, None)
    qml.DepolarizingChannel(p, 3, False, None)
    qml.DepolarizingChannel(p, 4, False, None)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3))

print(qml.draw(Depolarizing_circuit)(0.1))
dev_depolarizing._Depolarizing_circuit.draw(output="mpl")

# ps = [0.001, 0.01, 0.1, 0.2]
# for p in ps :
#     print(f"La sortie du QNode pour une probabilité de dépolarisation de {p} est {Depolarizing_circuit(p):.4f}")


0: ──H─┤ ╭<Z@Z@Z@Z>
1: ──H─┤ ├<Z@Z@Z@Z>
2: ──H─┤ ├<Z@Z@Z@Z>
3: ──H─┤ ╰<Z@Z@Z@Z>


AttributeError: 'DefaultMixed' object has no attribute '_Depolarizing_circuit'

## Optimisation des canaux - Channel optimization

A présent, supposons que nous exécutons le circuit pour préparer un état Bell sur un périphérique matériel et observons que la valeur d'attente n'est pas égal à $1$ (comme cela se produirait avec un appareil idéal), mais a plutôt la valeur $0,7781$.

Il est connu que dans cette expérience la source principale de bruit est *l'amortissement d'amplitude*, qui se définit par les opérateurs de Kraus :


$K_0 =   \left( {\begin{array}{cc}
    1 & 0 \\
    0 & \sqrt{1 - p} \\
  \end{array} } \right)$

$K_1 =   \left( {\begin{array}{cc}
    0 & \sqrt{p} \\
    0 & 0 \\
  \end{array} } \right)$


Nous cherchons ici à déterminer la probabilité $p$ pour laquelle l'expérience sur une machine bruyante donne un résultat de $0,7781$. 


\- \- \-


Now suppose we run the circuit to prepare a Bell state on a hardware device and observe that the expectation value does not equal $1$ (as it would with an ideal device), but instead has the value $0.7781$.

It is known that in this experiment the main source of noise is *amplitude damping*, which is defined by the Kraus operators:


$K_0 = \left( {\begin{array}{cc}
    1 & 0 \\
    0 & \sqrt{1 - p} \\
  \end{array} } \right)$

$K_1 = \left( {\begin{array}{cc}
    0 & \sqrt{p} \\
    0 & 0 \\
  \end{array} } \right)$


We seek here to determine the probability $p$ for which the experiment on a noisy machine gives a result of $0.7781$.


In [ ]:
# Définissons la valeur attendue
ev = np.tensor(0.7781, requires_grad = False)

# Définissons une fonction sigmoïde afin de traduire le résultat en une probabilité entre 0 et 1
def sigmoid(x):
    return  1 / (1 + np.exp(-x))

# Définissons notre circuit
@qml.qnode(dev)
def damping_circuit(x):
    qml.Hadamard(wires = 0)
    qml.CNOT(wires=[0,1])
    qml.AmplitudeDamping(sigmoid(x), wires=0)
    qml.AmplitudeDamping(sigmoid(x), wires=1)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))





Optimisons le circuit par rapport à une fonction de coût qui atteint son minimum lorsque la sortie QNode est égale à $0.7781$ :

\- \- \-

Let's optimize the circuit against a cost function that reaches its minimum when the QNode output is equal to $0.7781$:

In [ ]:
def cost(x, target):
    return (damping_circuit(x) - target)**2


A présent, utilisons une descente de gradient afin d'optimiser le paramètre :

\- \- \- 

Now let's use gradient descent to optimize the parameter:

In [ ]:
opt = qml.GradientDescentOptimizer(stepsize=10)
steps = 35
x = np.tensor(0.0, requires_grad=True)

for i in range(steps):
    (x, ev), cost_val = opt.step_and_cost(cost, x, ev)
    if i % 5 == 0 or i == steps - 1:
        print(f"Step: {i}    Cost: {cost_val}")


print(f"QNode output after optimization = {damping_circuit(x):.4f}")
print(f"Experimental expectation value = {ev}")
print(f"Optimized noise parameter p = {sigmoid(x.take(0)):.4f}")


c:\Users\maell\AppData\Local\Programs\Python\Python310\lib\site-packages\autograd\numpy\numpy_wrapper.py:156: ComplexWarning: Casting complex values to real discards the imaginary part
  return A.astype(dtype, order, casting, subok, copy)


Step: 0    Cost: 0.07733961000000007
Step: 5    Cost: 0.07733961000000007
Step: 10    Cost: 0.07733961000000007
Step: 15    Cost: 0.07733961000000007
Step: 20    Cost: 0.07733961000000007
Step: 25    Cost: 0.07733961000000007
Step: 30    Cost: 0.07733961000000007
Step: 34    Cost: 0.07733961000000007
QNode output after optimization = 0.5000
Experimental expectation value = 0.7781
Optimized noise parameter p = 0.5000
